# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV, KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor

## Part 1 - Data Preprocessing

### Importing the dataset

In [2]:
dataset = pd.read_excel('RPDataset1.xlsx')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588 entries, 0 to 587
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Explosive type                588 non-null    object 
 1   Explosive mass(kg)            588 non-null    float64
 2   Standoff distance(m)          588 non-null    float64
 3   Peak reflected pressure(kPa)  588 non-null    float64
dtypes: float64(3), object(1)
memory usage: 18.5+ KB


In [3]:
# convert categorical variable into dummy variables
dataset = pd.get_dummies(dataset, columns=['Explosive type'])
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588 entries, 0 to 587
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Explosive mass(kg)            588 non-null    float64
 1   Standoff distance(m)          588 non-null    float64
 2   Peak reflected pressure(kPa)  588 non-null    float64
 3   Explosive type_Composition B  588 non-null    uint8  
 4   Explosive type_TNT            588 non-null    uint8  
dtypes: float64(3), uint8(2)
memory usage: 15.1 KB


In [4]:
dataset.head()

,Explosive mass(kg),Standoff distance(m),Peak reflected pressure(kPa),Explosive type_Composition B,Explosive type_TNT
0,0.5,1.5,849.52040,1,0
1,0.5,10.5,116.73901,1,0
2,0.5,11.5,114.90842,1,0
3,0.5,12.5,113.43890,1,0
4,0.5,13.5,112.23654,1,0


In [5]:
y = dataset[('Peak reflected pressure(kPa)')]
X = dataset.drop('Peak reflected pressure(kPa)', axis=1)
print(X.shape, y.shape)

(588, 4) (588,)


In [6]:
print (X.head())

   Explosive mass(kg)  Standoff distance(m)  Explosive type_Composition B  \
0                 0.5                   1.5                             1   
1                 0.5                  10.5                             1   
2                 0.5                  11.5                             1   
3                 0.5                  12.5                             1   
4                 0.5                  13.5                             1   

   Explosive type_TNT  
0                   0  
1                   0  
2                   0  
3                   0  
4                   0  


In [7]:
y.head()

0    849.52040
1    116.73901
2    114.90842
3    113.43890
4    112.23654
Name: Peak reflected pressure(kPa), dtype: float64

In [8]:
# convert to numpy array
X = np.array(X)
y = np.array(y)

### Splitting the dataset into the Training set and Test set

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 71)

In [10]:
X_val, X_test, y_val, y_test = train_test_split(X_test,
                                                y_test,
                                                test_size = 0.5,
                                                random_state = 71)

### Feature scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:,0:2] = sc.fit_transform(X_train[:, 0:2])
print (X_train)

In [ ]:
X_test[:,0:2] = sc.transform(X_test[:, 0:2])
print (X_test)

In [ ]:
X_val[:,0:2] = sc.transform(X_val[:, 0:2])
print (X_val)

### S1 -Hyperparameter tuning - neurons, activation function

In [71]:
from tensorflow.keras.layers import Dropout
# Set seed for NumPy
np.random.seed(54)

# Set seed for TensorFlow
tf.random.set_seed(54)

from tensorflow.keras.optimizers import Adam, RMSprop, Nadam
def create_model1(last_layer_nodes, activation_func):
    model = Sequential()
    model.add(Dense(units = 250,  input_shape=(X_train.shape[1],), activation=activation_func))
    #model.add(Dropout(0.1))
    model.add(Dense(last_layer_nodes, activation=activation_func))
    #model.add(Dropout(0.1))
    model.add(Dense(1, activation='linear'))
    optimizer1 = Adam(learning_rate=0.01)
    model.compile(optimizer = optimizer1, loss = 'mean_squared_error', metrics = ['mae'])
    return model

activation_func = ['relu', 'leaky_relu', 'softplus']
last_layer_nodes = [50,60, 70, 80,90,100,110,120,130, 140, 150,160, 170, 180, 190, 200, 210, 220, 230, 240, 250]

param_grid = dict( model__activation_func = activation_func,model__last_layer_nodes = last_layer_nodes)

##Wrap model into scikit-learn
model1 = KerasRegressor(model=create_model1, verbose=0, epochs = 100, batch_size = 50, random_state = 54)

In [72]:
kf = KFold(n_splits=5, shuffle=True, random_state=54)
grid1 = GridSearchCV(estimator = model1, param_grid= param_grid, n_jobs=-1, scoring = 'r2', cv=kf)
grid_result1 = grid1.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result1.best_score_, grid_result1.best_params_))
means = grid_result1.cv_results_['mean_test_score']
stds = grid_result1.cv_results_['std_test_score']
params = grid_result1.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.965583 using {'model__activation_func': 'relu', 'model__last_layer_nodes': 210}
0.931790 (0.066760) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 50}
0.947392 (0.044960) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 60}
0.944336 (0.050107) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 70}
0.954378 (0.026744) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 80}
0.952455 (0.028046) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 90}
0.928899 (0.057181) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 100}
0.960983 (0.020133) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 110}
0.955239 (0.026119) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 120}
0.959856 (0.020881) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 130}
0.957367 (0.022299) with: {'model__activation_func': 'relu', 'model__last_layer_

In [73]:
pd.concat([pd.DataFrame(params), pd.DataFrame({'model__first_layer_nodes': [250] * len(params)}),pd.DataFrame(means, columns=['R2'])], axis =1)

,model__activation_func,model__last_layer_nodes,model__first_layer_nodes,R2
0,relu,50,250,0.931790
1,relu,60,250,0.947392
2,relu,70,250,0.944336
3,relu,80,250,0.954378
4,relu,90,250,0.952455
...,...,...,...,...
58,softplus,210,250,0.949144
59,softplus,220,250,0.949310
60,softplus,230,250,0.950075
61,softplus,240,250,0.950537


### Merge all files

In [74]:
from pathlib import Path

current_dir = Path.cwd()
redirect_dir = current_dir.parent.parent.parent

#Access input folder
input_dir = Path (redirect_dir/"1D_Tracers/Hyperparameter tuning summary_ANN/Reflected_pressure_hyperparameter_tuning2")
print ("1",input_dir)

# Output Excel file
output_excel_file = Path(input_dir/"S1_summary.xlsx")

# List to store DataFrames from CSV files
dfs = []

# Loop through CSV files in the directory
for csv_file in input_dir.glob('*.csv'):
    # Read CSV file into a DataFrame and append to the list
    df = pd.read_csv(csv_file, header = None)
    dfs.append(df)

print (dfs)

# Concatenate DataFrames in the list along rows
merged_df = pd.concat(dfs, axis = 0,ignore_index=False)

# Write the merged DataFrame to an Excel file
merged_df.to_excel(output_excel_file, index=False)

1 G:\Chamodi\1D_Tracers\Hyperparameter tuning summary_ANN\Reflected_pressure_hyperparameter_tuning2
[     0         1    2   3         4
0    0      relu   50  50  0.866445
1    1      relu   60  50  0.879698
2    2      relu   70  50  0.898776
3    3      relu   80  50  0.883987
4    4      relu   90  50  0.897611
..  ..       ...  ...  ..       ...
58  58  softplus  210  50  0.927736
59  59  softplus  220  50  0.930607
60  60  softplus  230  50  0.928415
61  61  softplus  240  50  0.933343
62  62  softplus  250  50  0.932596

[63 rows x 5 columns],      0         1    2    3         4
0    0      relu   50  140  0.924321
1    1      relu   60  140  0.917221
2    2      relu   70  140  0.914532
3    3      relu   80  140  0.921420
4    4      relu   90  140  0.925865
..  ..       ...  ...  ...       ...
58  58  softplus  210  140  0.937001
59  59  softplus  220  140  0.943296
60  60  softplus  230  140  0.946155
61  61  softplus  240  140  0.956641
62  62  softplus  250  140  0.962699

### S2 - Hyperparameter tuning - batch size, epoch, optimizer, learning rate

In [75]:
from tensorflow.keras.optimizers import RMSprop, Nadam
# Set seed for NumPy
np.random.seed(54)

# Set seed for TensorFlow
tf.random.set_seed(54)

# Function to create model, required for KerasClassifier
def create_model2():
 # create model
 model = Sequential()
 model.add(Dense(units=240, input_shape=(X_train.shape[1],), activation='relu'))
 #model.add(Dropout(0.1))
 model.add(Dense(units=190, activation='relu'))
 #model.add(Dropout(0.1))
 model.add(Dense(units=1, activation='linear'))

 return model

In [76]:
# create model
model2 = KerasRegressor(model=create_model2, verbose=0, random_state = 54, loss = 'mean_squared_error', metrics = ['mae'])

# define the grid search parameters
batch_size = [30,40,50]
optimizer = [Adam, Nadam, RMSprop]
learning_rate = [ 0.001,0.01, 0.1, 1, 2]
epochs = [500]

# gridsearch
param_grid2 = dict(batch_size=batch_size, optimizer=optimizer, optimizer__learning_rate = learning_rate, epochs = epochs)
kf = KFold(n_splits=5, shuffle=True, random_state=54)
grid2 = GridSearchCV(estimator=model2, param_grid=param_grid2, n_jobs=-1, scoring = 'r2', cv=kf)
grid_result2 = grid2.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result2.best_score_, grid_result2.best_params_))
means = grid_result2.cv_results_['mean_test_score']
stds = grid_result2.cv_results_['std_test_score']
params = grid_result2.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.996012 using {'batch_size': 40, 'epochs': 500, 'optimizer': <class 'keras.src.optimizers.nadam.Nadam'>, 'optimizer__learning_rate': 0.1}
0.939661 (0.045601) with: {'batch_size': 30, 'epochs': 500, 'optimizer': <class 'keras.src.optimizers.adam.Adam'>, 'optimizer__learning_rate': 0.001}
0.989652 (0.010449) with: {'batch_size': 30, 'epochs': 500, 'optimizer': <class 'keras.src.optimizers.adam.Adam'>, 'optimizer__learning_rate': 0.01}
0.986814 (0.009287) with: {'batch_size': 30, 'epochs': 500, 'optimizer': <class 'keras.src.optimizers.adam.Adam'>, 'optimizer__learning_rate': 0.1}
0.303029 (0.395239) with: {'batch_size': 30, 'epochs': 500, 'optimizer': <class 'keras.src.optimizers.adam.Adam'>, 'optimizer__learning_rate': 1}
0.037679 (0.092810) with: {'batch_size': 30, 'epochs': 500, 'optimizer': <class 'keras.src.optimizers.adam.Adam'>, 'optimizer__learning_rate': 2}
0.942037 (0.040403) with: {'batch_size': 30, 'epochs': 500, 'optimizer': <class 'keras.src.optimizers.nadam.Nadam'>,

In [77]:
pd.concat([pd.DataFrame(params), pd.DataFrame(means, columns=['R2'])], axis =1)

,batch_size,epochs,optimizer,optimizer__learning_rate,R2
0,30,500,<class 'keras.src.optimizers.adam.Adam'>,0.001,0.939661
1,30,500,<class 'keras.src.optimizers.adam.Adam'>,0.010,0.989652
2,30,500,<class 'keras.src.optimizers.adam.Adam'>,0.100,0.986814
3,30,500,<class 'keras.src.optimizers.adam.Adam'>,1.000,0.303029
4,30,500,<class 'keras.src.optimizers.adam.Adam'>,2.000,0.037679
5,30,500,<class 'keras.src.optimizers.nadam.Nadam'>,0.001,0.942037
6,30,500,<class 'keras.src.optimizers.nadam.Nadam'>,0.010,0.988816
7,30,500,<class 'keras.src.optimizers.nadam.Nadam'>,0.100,0.976378
8,30,500,<class 'keras.src.optimizers.nadam.Nadam'>,1.000,0.289791
9,30,500,<class 'keras.src.optimizers.nadam.Nadam'>,2.000,-0.016463


In [78]:
# Get the best hyperparameters from the grid search
best_params = grid_result2.best_params_
print (best_params)

{'batch_size': 40, 'epochs': 500, 'optimizer': <class 'keras.src.optimizers.nadam.Nadam'>, 'optimizer__learning_rate': 0.1}


In [86]:
# Create the final model using the best hyperparameters
final_model = KerasRegressor(
    build_fn=create_model2,
    verbose=0,
    random_state=71,
    loss='mean_squared_error',
    metrics=['mae'],
    optimizer = Adam,
    optimizer__learning_rate = 0.1
    )

In [87]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss',
                   mode='min',
                   patience=50,
                   restore_best_weights = True)
# Train the final model on the entire training set
history = final_model.fit(X_train, y_train,
                validation_data = (X_val, y_val),
                callbacks=[es],
                epochs=best_params['epochs'],
                batch_size=best_params['batch_size'],
                verbose=1)

Epoch 1/500


D:\Anaconda3\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


9/9 [==============================] - 1s 44ms/step - loss: 55832392.0000 - mean_absolute_error: 3198.1384 - val_loss: 61575016.0000 - val_mean_absolute_error: 2673.4590
Epoch 2/500
9/9 [==============================] - 0s 9ms/step - loss: 42599968.0000 - mean_absolute_error: 2141.0654 - val_loss: 36317188.0000 - val_mean_absolute_error: 3169.7666
Epoch 3/500
9/9 [==============================] - 0s 8ms/step - loss: 27840188.0000 - mean_absolute_error: 1876.9525 - val_loss: 20953434.0000 - val_mean_absolute_error: 1656.2485
Epoch 4/500
9/9 [==============================] - 0s 9ms/step - loss: 17600802.0000 - mean_absolute_error: 1265.4921 - val_loss: 12577524.0000 - val_mean_absolute_error: 1069.9705
Epoch 5/500
9/9 [==============================] - 0s 8ms/step - loss: 8313207.0000 - mean_absolute_error: 792.6152 - val_loss: 8627887.0000 - val_mean_absolute_error: 1009.2715
Epoch 6/500
9/9 [==============================] - 0s 10ms/step - loss: 4699439.0000 - mean_absolute_error: 6

### Analyse learn history

In [88]:
# let's see the training and validation accuracy by epoch
history_dict = history.history
loss_values = history_dict['loss'] # you can change this
val_loss_values = history_dict['val_loss'] # you can also change this
epochs = range(1, len(loss_values) + 1) # range of X (no. of epochs)
plt.plot(epochs, loss_values, 'blue', label='Train set loss')
plt.plot(epochs, val_loss_values, 'orange', label='Validation set loss')
#plt.title('Training and testing loss')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.show()

AttributeError: 'KerasRegressor' object has no attribute 'history'

In [89]:
loss_values1 = history_dict['mae'] # you can change this
val_loss_values1 = history_dict['val_mae'] # you can also change this
epochs = range(1, len(loss_values1) + 1) # range of X (no. of epochs)
plt.plot(epochs, loss_values1, 'blue', label='Train set MAE')
plt.plot(epochs, val_loss_values1, 'orange', label='Validation set MAE')
#plt.title('Training and testing MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()

NameError: name 'history_dict' is not defined